In [2]:
from clean_tabular_data import *
import pandas as pd
#load dataframe with products from csv
products_df = pd.read_csv('Products.csv',lineterminator ='\n')  
# drop nulls and cast pricing to numeric  
products_df = clean_product_data(products_df)

In [3]:
def get_encoder_and_decoder(products_df):
        unique_categories = list(products_df["label"].unique())
        encoder = {k: v for v,k in enumerate(unique_categories)}
        decoder = {v: k for v,k in enumerate(unique_categories)}
        return (encoder,decoder)

# create new label column
products_df["label"] = products_df["category"].str.split(r"\/", expand=True)[0].str.strip()
#get encoder and decoder
encoder,decoder = get_encoder_and_decoder(products_df)
#save decoder to file
f = open("image_decoder.pkl","w")
f.write(str(decoder))
f.close()

In [3]:
images_df = pd.read_csv('Images.csv',lineterminator ='\n')
# merge images and products to get new label column for images
training_df = pd.merge(images_df, products_df[['id', 'label']], left_on='product_id', right_on='id', how='left', suffixes=('','_y')).drop('id_y', axis=1).drop('product_id', axis=1)
training_df = training_df.iloc[: , 1:]
training_df['label'] = training_df['label'].replace(encoder)
training_df.to_csv('training_data.csv')

In [1]:
from clean_images import *
# path specific to my local machine
clean_image_data('D:/Documents/AICore/images_fb/images')

In [5]:
import torch
import os
from datetime import datetime
from torch.utils.data import DataLoader, random_split
from FBMClassifier import FBMClassifier
from FBMDataset import FBMDataset
import torch.nn.functional as F
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms


def create_model_dir_path():
    parent_dir = 'model_evaluation'
    current_datetime = datetime.now().strftime('%y%m%d%H%M%S')
    child_dir = 'weights'
    path = os.path.join(os.getcwd(), parent_dir, current_datetime, child_dir)
    return path

def train(model,epochs):    
    writer = SummaryWriter()
    batch_id = 0

    path = create_model_dir_path()     
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Set the model to run on the device
    model = model.to(device)
    optimiser = SGD(model.parameters(), lr = 0.001)
    os.makedirs(path)

    for epoch in range(epochs):   
        model.train(True)     
        for batch in train_loader:
            # get features and labels from the batch
            features,labels = batch
            features = features.to(device)
            labels = labels.to(device)
            # loss.backward does not overwrite, it adds. To stop this, we set the gradients back to zero. sets the .grad of all optimized tensors to zero
            optimiser.zero_grad()
            # make a prediction
            prediction = model(features)
            # calculate loss
            criterion = F.cross_entropy(prediction,labels)
            # backward function calculates the gradient of the current tensor w.r.t graph leaves
            criterion.backward()
            # print(criterion.item())
            # moves each parameter in the opposite direction of the gradient, proportional to the learning rate
            optimiser.step()
            writer.add_scalar('Loss', criterion.item(), batch_id)
            batch_id += 1

        
        # Set the model to evaluation mode
        model.eval()
        running_vloss = 0.0
        
        with torch.no_grad():
            for i, vdata in enumerate(val_loader):
                vinputs, vlabels = vdata                
                vinputs = vinputs.to(device)
                vlabels = vlabels.to(device)
                
                voutputs = model(vinputs)
                vloss = F.cross_entropy(voutputs, vlabels)
                running_vloss += vloss

        avg_vloss = running_vloss / (i + 1)
        print(f'Epoch {epoch} - Average Loss: {avg_vloss}')
        torch.save(model.state_dict(), path + f'/epoch_{epoch}.pt')

#Steps for transfer learning: get pre trained model, change architecture to fit our problem, fine tune for our problem
classifer = FBMClassifier()
transform_list = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
dataset = FBMDataset("training_data.csv","../images_fb/images/cleaned_images",transform=transform_list)

#obtain the list of targets
train_dataset,test_dataset,val_dataset = random_split(dataset, [0.7,0.15,0.15])

train_loader = DataLoader(train_dataset,batch_size=4,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=4,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=4,shuffle=True)

train(classifer, 3)

Using cache found in C:\Users\user/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [3]:
import torch
device

device(type='cuda')